In [10]:
import pandas as pd
import numpy as np
import os 

file_name=input("Enter File Name: ")
journey="up"
file_path=os.path.join('week_data',journey, file_name)

if not os.path.isfile(file_path):
    print(f"Error: File '{file_name}' does not exist.")
else:
    df=pd.read_csv(file_path)

bus_no = int(input("Enter bus Number: "))
df = df[df['Schedule Name'].str.contains(rf'\b{bus_no}\b')]

cols=['Schedule Name','Ticket Issued Time','Adult','From Stage','To Stage','Trip Start Time','Trip End Time','Source','Destination']
df=df[cols]

source_stage=input("Enter source: ").upper()
destination_stage=input("Enter destination: ").upper()
df=df[(df['Source']==source_stage) & (df['Destination']==destination_stage)]

df['Ticket Issued Time'] = pd.to_datetime(df['Ticket Issued Time'], format='%H:%M:%S')

start=input("Enter Start time: ")
end=input("Enter End time: ")
start_time = pd.to_datetime(start).time()
end_time = pd.to_datetime(end).time()

df = df[(df['Ticket Issued Time'].dt.time >= start_time) & (df['Ticket Issued Time'].dt.time < end_time)]

bus_stages = [
    'T.NAGAR', 'SAIDAPET','CONCORDE', 'ANNA UNIV', 'WPTC', 'SRP TOOLS',
    'KANDANCHAV', 'THORAIPAKKAM', 'M K CHAVADI', 'KARAPAKKAM',
    'SHOLINGANALLUR', 'KUMARAN NG', 'CHEMMANCHE', 'NAVALUR',
    'SIPCOT', 'CHURCH', 'PAL. CHEMI', 'HINDUSTAN', 'KELAMBAKKAM',
    'KOMAN NAGAR', 'ENGG', 'CHENGAMMAL', 'KALAVAKKAM','THIRUPORUR'
]


stage_mapping = {stage: i for i, stage in enumerate(bus_stages)}

df['From Stage'] = df['From Stage'].map(stage_mapping)
df['To Stage'] = df['To Stage'].map(stage_mapping)
df.dropna(subset=['From Stage', 'To Stage'], inplace=True)

# Create an empty OD matrix with stage names as indexes and columns
odm_up = pd.DataFrame(index=bus_stages, columns=bus_stages).fillna(0)

# Populate the matrix 
for index, row in df.iterrows():
    source = bus_stages[int(row['From Stage'])]
    destination = bus_stages[int(row['To Stage'])]
    adult_count = row['Adult']
    if not pd.isna(adult_count):
        odm_up.loc[source, destination] += adult_count
    else:
        pass

odm_up['Boarding'] = odm_up.sum(axis=1)
odm_up.loc['ALIGHTING', :] = odm_up.sum(axis=0)

Enter File Name: 05-02-19.csv
Enter bus Number: 519
Enter source: T.NAGAR
Enter destination: THIRUPORUR
Enter Start time: 07:00:00
Enter End time: 08:00:00


In [11]:
journey="down"
file_path=os.path.join('week_data',journey, file_name)

if not os.path.isfile(file_path):
    print(f"Error: File '{file_name}' does not exist.")
else:
    df=pd.read_csv(file_path)

df = df[df['Schedule Name'].str.contains(rf'\b{bus_no}\b')]

cols=['Schedule Name','Ticket Issued Time','Adult','From Stage','To Stage','Trip Start Time','Trip End Time','Source','Destination']
df=df[cols]
df=df[(df['Source']==destination_stage) & (df['Destination']==source_stage)]

df['Ticket Issued Time'] = pd.to_datetime(df['Ticket Issued Time'], format='%H:%M:%S')
df = df[(df['Ticket Issued Time'].dt.time >= start_time) & (df['Ticket Issued Time'].dt.time < end_time)]


stage_mapping = {stage: i for i, stage in enumerate(bus_stages)}

df['From Stage'] = df['From Stage'].map(stage_mapping)
df['To Stage'] = df['To Stage'].map(stage_mapping)
df.dropna(subset=['From Stage', 'To Stage'], inplace=True)

odm_down = pd.DataFrame(index=bus_stages, columns=bus_stages).fillna(0)

for index, row in df.iterrows():
    source = bus_stages[int(row['From Stage'])]
    destination = bus_stages[int(row['To Stage'])]
    adult_count = row['Adult']
    if not pd.isna(adult_count):
        odm_down.loc[source, destination] += adult_count
    else:
        pass

odm_down['Boarding'] = odm_down.sum(axis=1)
odm_down.loc['ALIGHTING', :] = odm_down.sum(axis=0)

In [12]:
result_matrix = odm_up + odm_down
result_matrix

,T.NAGAR,SAIDAPET,CONCORDE,ANNA UNIV,WPTC,SRP TOOLS,KANDANCHAV,THORAIPAKKAM,M K CHAVADI,KARAPAKKAM,...,CHURCH,PAL. CHEMI,HINDUSTAN,KELAMBAKKAM,KOMAN NAGAR,ENGG,CHENGAMMAL,KALAVAKKAM,THIRUPORUR,Boarding
T.NAGAR,0.0,0.0,0.0,0.0,0.0,3.0,7.0,1.0,3.0,4.0,...,8.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0
SAIDAPET,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,1.0,...,2.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,14.0
CONCORDE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ANNA UNIV,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0
WPTC,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0
SRP TOOLS,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
KANDANCHAV,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0
THORAIPAKKAM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
M K CHAVADI,5.0,4.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0
KARAPAKKAM,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0


In [14]:
store_file=input("Enter store file ")
result_matrix.to_excel(f"{store_file}.xlsx")

Enter store file odm_combine_tickettime
